## Project: Dashboard (Analyze The Northwind Dataset)

#### Workflow
1. create a Postgres database
2. Download the data from GitHub.
3. create the table definitions
4. use the SQL COPY command to load the CSV files into the database
5. write SQL queries to retrieve data by answering the questions below
6.   load CSV files into a database
7.  use the sqlalchemy library to access a db from Python
8. build a dashboard
9.   upload everything into a cloud server

### Step 5 - SQL queries

In [ ]:
#1. Get the names and the quantities in stock for each product.
SELECT productName, quantityPerUnit 
FROM Products;

In [ ]:
#2. Get a list of current products (Product ID and name).

#(Method 1)
SELECT ProductID, ProductName 
FROM Products WHERE Discontinued = False; 

#(Method 2)
SELECT ProductID, ProductName 
FROM Products WHERE Discontinued = 0; 

In [ ]:
#3. Get a list of the most and least expensive products (name and unit price).
SELECT productName, unitPrice FROM products ORDER BY unitPrice DESC;
SELECT productName, unitPrice FROM products WHERE unitPrice > 200 OR unitPrice < 3 ORDER BY unitPrice DESC;
SELECT MAX(UnitPrice), MIN(UnitPrice) FROM Products;

In [ ]:
#4. Get products that cost less than $20.

#(Method 1)
SELECT productID, productName, unitPrice 
FROM products WHERE unitPrice<20

#(Method 2)
SELECT ProductID, ProductName, UnitPrice 
FROM Products WHERE (((UnitPrice)<20) AND ((Discontinued)=False)) 
ORDER BY UnitPrice DESC;

In [ ]:
#5. Get products that cost between $15 and $25.

#(Method 1)
SELECT ProductID, ProductName, UnitPrice 
FROM Products WHERE UnitPrice >=15 AND UnitPrice <=25 
ORDER BY UnitPrice DESC; 

#(Method 2)
SELECT ProductID, ProductName, UnitPrice 
FROM Products WHERE UnitPrice >=15 AND UnitPrice <=25  AND Discontinued=0 
ORDER BY UnitPrice DESC;

In [ ]:
#6. Get products above average price.

SELECT productName, unitPrice 
FROM products WHERE UnitPrice > (SELECT avg(UnitPrice) FROM Products) 
ORDER BY unitPrice DESC;

In [ ]:
#7. Find the ten most expensive products.

#(Method 1)
SELECT productName, unitPrice 
FROM products 
ORDER BY unitPrice DESC LIMIT 10; 

#(Method 2)
SELECT DISTINCT ProductName as Ten_Most_Expensive_Products, unitPrice 
FROM products 
ORDER BY unitPrice DESC LIMIT 10; 

In [ ]:
#8. Get a list of discontinued products (Product ID and name).
SELECT ProductID, ProductName 
FROM Products WHERE Discontinued = 1;

In [ ]:
#9. Count current and discontinued products.
SELECT Count(productID) 
FROM products GROUP BY Discontinued;

In [ ]:
#10. Find products with less units in stock than the quantity on order.

#(Method 1)
SELECT productName,  unitsOnOrder, UnitsInStock 
FROM products WHERE unitsInStock < unitsOnOrder; 

#(Method 2)
SELECT productName,  unitsOnOrder, UnitsInStock 
FROM products WHERE unitsInStock < unitsOnOrder AND discontinued=0; 

In [ ]:
#11. Find the customer who had the highest order amount
SELECT customer_id, COUNT(*) 
FROM orders GROUP BY customer_id 
ORDER BY orders DESC LIMIT 1;

In [ ]:
#12. Get orders for a given employee and the according customer
SELECT employees.lastname, customers.companyname, orders.orderid
FROM orders JOIN employees ON orders.employeeid = employees.employeeid
JOIN customers ON orders.customerid = customers.customerid ORDER BY lastname, companyname;

In [ ]:
#13. Find the hiring age of each employee

#(1. gives answer in days)
SELECT CONCAT(titleOfCourtesy, lastName, ' ', firstName) AS fullName,  hireDate - birthDate as hiringAGE FROM employees; 

#(2. gives answer in y,m,d)
select CONCAT(titleOfCourtesy, lastName, ' ', firstName) AS fullName, age(hiredate, birthdate) from employees; 

#(Method 3)
select CONCAT(titleOfCourtesy, lastName, ' ', firstName) AS fullName, extract(year from age(hiredate, birthdate)) as hireage from employees; 

In [ ]:
#14. Calculate employee revenue
SELECT e.firstName, e.lastName, SUM(od.unit_price*od.quantity) AS revenue 
FROM orders As o JOIN employees AS e ON o.employeeID=e.employeeID
JOIN order_details AS od ON o.orderID=od.orderID GROUP BY e.firstName, e.lastName 
ORDER BY revenue DESC;

In [ ]:
#15. Create a view of all the relevant columns from all the tables
CREATE VIEW all_data AS
    (SELECT categories.category_id, categories.category_name,
        products.product_id, products.product_name,products.unit_price,
        order_details.order_id, order_details.quantity,
        orders.customer_id, orders.order_date, orders.ship_country, orders.ship_city,
        customers.city, customers.country
    FROM categories
    JOIN products on products.category_id = categories.category_id
    JOIN order_details on order_details.product_id = products.product_id
    JOIN orders on orders.order_id = order_details.order_id
    JOIN customers on customers.customer_id = orders.customer_id);

In [ ]:
#16. Mark products as expensive of inexpensive
SELECT product_id,
CASE
    WHEN unit_price > 150 THEN 'expensive'
    ELSE 'inexpensive' END
FROM order_details;

In [ ]:
#17. Calculate the average order value per country
SELECT
    AVG(order_details.product_id * order_details.unit_price - order_details.discount) AS order_value,
    orders.ship_country
FROM order_details
JOIN orders on orders.order_id = order_details.order_id
GROUP BY orders.ship_country
ORDER BY order_value DESC;

In [ ]:
#18. Calculate the average order value by date
SELECT
    AVG(order_details.product_id * order_details.unit_price - order_details.discount) AS order_value,
    orders.order_date
FROM order_details
JOIN orders on orders.order_id = order_details.order_id
GROUP BY orders.order_date
ORDER BY order_value DESC;